<a href="https://colab.research.google.com/github/flora0110/LLMRec-Labs-Colab/blob/main/STF_tuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pakage install

# PATH

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

GDRIVE_ROOT = "/content/drive/My Drive/TAMU_LAB"
DATA_DIR    = f"{GDRIVE_ROOT}/data"
EXP_DIR     = f"{GDRIVE_ROOT}/experiences/models"

GOODREADS_SAMPLED = f"{DATA_DIR}/sampled/Goodreads"
GOODREADS_EVAL    = f"{DATA_DIR}/eval/Goodreads"

run_name = "SFT_run_colab_1"
OUTPUT_DIR = f"{EXP_DIR}/{run_name}"

print("DATA_DIR:   ", DATA_DIR)
print("SAVE TO:    ", OUTPUT_DIR)


Mounted at /content/drive
DATA_DIR:    /content/drive/My Drive/TAMU_LAB/data
SAVE TO:     /content/drive/My Drive/TAMU_LAB/experiences/models/SFT_run_colab_1


In [ ]:
!pip install -U transformers

In [ ]:
!pip install trl==0.16.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 7.8 MB/s eta 0:00:00


In [54]:
from huggingface_hub import notebook_login
notebook_login()

## Local Inference on GPU
Model page: https://huggingface.co/Qwen/Qwen3-0.6B

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/Qwen/Qwen3-0.6B)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="Qwen/Qwen3-0.6B")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': '<think>\nOkay, the user asked, "Who are you?" I need to respond appropriately. First, I should acknowledge their question. I can say, "Hello! I\'m your virtual assistant, and I\'m here to help you with all your questions." Then, I should mention that I can assist with various tasks, like answering questions or providing information. I should keep it friendly and open-ended to encourage further interaction. Let me make sure the response is clear and polite.\n</think>\n\nHello! I\'m your virtual assistant, and I\'m here to help you with all your questions. How can I assist you today?'}]}]

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

<think>
Okay, the user asked, "Who are you?" So I need to respond appropriately. First, I should acknowledge their question and then explain who I am. Since I'm an AI assistant


# Utils

In [ ]:
import os, json

def safe_write_json(file_path, data, indent=2):
    directory = os.path.dirname(file_path)
    if directory and not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)

    base_path, ext = os.path.splitext(file_path)
    final_path = file_path
    counter = 1
    while os.path.exists(final_path):
        print(f"Warning: {final_path} already exists. Generating new file name...")
        final_path = f"{base_path}_{counter}{ext}"
        counter += 1

    with open(final_path, "w") as f:
        json.dump(data, f, indent=indent, ensure_ascii=False)
    print(f"Saved data to {final_path}")

def safe_load_json(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    with open(file_path, "r") as f:
        return json.load(f)

def prepare_output_dir(output_path: str, check_subdir: str = "final_model", allow_existing: bool = False) -> str:
    if not os.path.exists(output_path):
        os.makedirs(output_path, exist_ok=True)
        print(f"Created base output dir: {output_path}")

    if allow_existing:
        if check_subdir is None:
            print(f"Using existing output directory: {output_path}")
            return output_path
        else:
            subdir = os.path.join(output_path, check_subdir)
            if not os.path.exists(subdir):
                os.makedirs(subdir, exist_ok=True)
                print(f"Created subdirectory: {subdir}")
            else:
                print(f"Using existing subdirectory: {subdir}")
            return subdir

    if check_subdir is None:
        if len(os.listdir(output_path)) == 0:
            print(f"Warning: Output directory '{output_path}' exists and is empty. No need to create a new directory.")
            return output_path
        else:
            base_path = output_path
            counter = 1
            final_output_path = base_path
            while os.path.exists(final_output_path) and os.listdir(final_output_path):
                print(f"Warning: '{final_output_path}' already exists and is not empty. Generating a new output directory name...")
                final_output_path = f"{base_path}_{counter}"
                counter += 1
            os.makedirs(final_output_path, exist_ok=True)
            print(f"Using output directory: {final_output_path}")
            return final_output_path
    else:
        final_model_dir = os.path.join(output_path, check_subdir) if check_subdir else output_path
        counter = 1
        final_output_path = output_path
        while os.path.exists(final_model_dir):
            print(f"Warning: '{final_model_dir}' already exists. Generating a new output directory name...")
            final_output_path = f"{output_path}_{counter}"
            final_model_dir = os.path.join(final_output_path, check_subdir) if check_subdir else final_output_path
            counter += 1
        os.makedirs(final_output_path, exist_ok=True)
        print(f"Using output directory: {final_output_path}")
        return final_output_path


# SFT Trainer

In [55]:
import os, torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM

def load_model(base_model_name, use_4bit=True, local_files_only=False):
    print("\nLoading model:", base_model_name)
    quant_cfg = None
    if use_4bit:
        quant_cfg = BitsAndBytesConfig(
            load_in_4bit=True, # 4 to 8
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=False,
        )
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        # quantization_config=quant_cfg, # remove bitsandbytes
        torch_dtype=torch.float16,
        device_map="auto",
        local_files_only=local_files_only
    )
    tokenizer = AutoTokenizer.from_pretrained(
        base_model_name,
        local_files_only=local_files_only
    )
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    response_template_with_context = "\n### Response:\n"  # We added context here: "\n". This is enough for this tokenizer
    response_template_ids = tokenizer.encode(response_template_with_context, add_special_tokens=False)[2:]  # Now we have it like in the dataset texts: `[2277, 29937, 4007, 22137, 29901]`

    collator = DataCollatorForCompletionOnlyLM(
        response_template_ids,
        tokenizer=tokenizer,
    )

    model.config.use_cache = False
    if use_4bit:
        model = prepare_model_for_kbit_training(model)
    return model, tokenizer, collator

def add_lora(model, lora_params):
    print("\nApplying LoRA")
    cfg = LoraConfig(
        r=lora_params.get("r", 16),
        lora_alpha=lora_params.get("lora_alpha", 32),
        target_modules=lora_params.get("target_modules", ['q_proj', 'v_proj']),
        lora_dropout=lora_params.get("lora_dropout", 0.0),
        bias=lora_params.get("bias", "none"),
        task_type=lora_params.get("task_type", "CAUSAL_LM"),
    )
    model = get_peft_model(model, cfg)
    model.print_trainable_parameters()
    return model
def formatting_prompts_func(example):
    instruction = example["instruction"]
    input_text = example.get("input", "")
    response = example["output"].strip()
    formatted = f"### Instruction:\n{instruction}\n"
    if input_text and input_text.strip():
        formatted += f"### Input:\n{input_text}\n"
    formatted += f"### Response:\n{response}</s>"
    return formatted

def train_model(model, collator, train_data, val_data, training_params, output_dir):
    print("\nStarting training...")
    sft_args = SFTConfig(
        output_dir=output_dir,
        max_length=2048,
        per_device_train_batch_size=training_params.get("per_device_train_batch_size", 2),
        per_device_eval_batch_size=training_params.get("per_device_eval_batch_size", 2),
        gradient_accumulation_steps=training_params.get("gradient_accumulation_steps", 2),
        warmup_steps=training_params.get("warmup_steps", 20),
        learning_rate=float(training_params.get("learning_rate", 2e-5)),
        num_train_epochs=training_params.get("num_train_epochs", 3),
        logging_steps=training_params.get("logging_steps", 10),
        bf16=training_params.get("bf16", True),
        optim=training_params.get("optim", "adamw_torch"),
        eval_strategy="steps",
        eval_steps=training_params.get("eval_steps", 100),
        save_steps=training_params.get("save_steps", 200),
        save_total_limit=training_params.get("save_total_limit", 2),
        report_to=training_params.get("report_to", []),
        push_to_hub=True, # 自動上傳
        hub_model_id="chuanhsin0110/Goodreads-SFT", # 換成你的 repo 名稱
        hub_strategy="checkpoint",
    )
    trainer = SFTTrainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=val_data,
        args=sft_args,
        formatting_func=formatting_prompts_func,
        data_collator=collator,
    )
    trainer.train()
    final_output_dir = os.path.join(output_dir, "final_model")
    trainer.model.save_pretrained(final_output_dir, safe_serialization=False)
    trainer.tokenizer.save_pretrained(final_output_dir)
    print("Training completed, model saved to:", final_output_dir)

def print_tokens_with_ids(txt):
    tokens = tokenizer.tokenize(txt, add_special_tokens=False)
    token_ids = tokenizer.encode(txt, add_special_tokens=False)
    print(list(zip(tokens, token_ids)))

def sanity_check(samples):
    response_template = "### Response:"
    bad = []
    for i, s in enumerate(samples[:10]):
        t = formatting_prompts_func(s)
        print_tokens_with_ids(t)
        print_tokens_with_ids(response_template)
        #print(len(t))
        if t.count("### Response:") != 1:
            bad.append(i)
        elif len(t) > 2048:  # 依你設定的 max_seq_length
          bad.append(i)
    if bad:
        print(f"[WARN] {len(bad)} / 100 samples have 0 or multiple '### Response:\\n' → {bad[:10]}")


def train_sft(config):
    torch.manual_seed(config.get("seed", 0))

    base_model_name = config["base_model_name"]
    output_path     = prepare_output_dir(config["output_path"])
    train_data_path = config["train_data_path"]
    valid_data_path = config["valid_data_path"]

    print("Torch CUDA:", torch.version.cuda)
    print("Output dir:", output_path)

    model, tokenizer, collator = load_model(
        base_model_name,
        use_4bit=config.get("use_4bit", True),
        local_files_only=config.get("local_files_only", False)
    )

    print("Loading datasets…")
    train_list = safe_load_json(train_data_path)
    #sanity_check(train_list)
    train_data = Dataset.from_list(train_list)

    val_list = safe_load_json(valid_data_path)
    #sanity_check(val_list)
    val_data = Dataset.from_list(val_list)

    # train_data = Dataset.from_list(safe_load_json(train_data_path))
    # val_data   = Dataset.from_list(safe_load_json(valid_data_path))

    print("Applying LoRA…")
    model = add_lora(model, config.get("lora", {}))

    print("Commencing training…")
    train_model(model, collator, train_data, val_data, config.get("training", {}), output_path)


# config

In [ ]:
# ---- 選一個 base 模型（可改成你要的）----
BASE_MODEL = "Qwen/Qwen3-0.6B"   # 舉例：你之前提過的
# 如果你已經把 base 模型放到 Drive，本地載入可將 local_files_only=True 並填本地路徑

# ---- 指定 train / valid 的實際檔案 ----
TRAIN_JSON = f"{GOODREADS_SAMPLED}/train.json"
VALID_JSON = f"{GOODREADS_SAMPLED}/valid.json"

config = {
    "seed": 0,
    "base_model_name": BASE_MODEL,
    "output_path": OUTPUT_DIR,              # 存到 My Drive/TAMU_LAB/experiences/models/<run_name>
    "train_data_path": TRAIN_JSON,
    "valid_data_path": VALID_JSON,
    "use_4bit": True,
    "local_files_only": False,              # 若 base 模型不在本機，讓 HF 下載
    "lora": {
        "r": 16,
        "lora_alpha": 32,
        "target_modules": ["q_proj", "v_proj"],
        "lora_dropout": 0.05,
        "bias": "none",
        "task_type": "CAUSAL_LM",
    },
    "training": {
        "per_device_train_batch_size": 2,
        "per_device_eval_batch_size": 2,
        "gradient_accumulation_steps": 2,
        "warmup_steps": 20,
        "learning_rate": 2e-5,
        "num_train_epochs": 3,
        "logging_steps": 5,
        "bf16": True,
        "optim": "adamw_torch",
        "eval_steps": 100,
        "save_steps": 200,
        "save_total_limit": 2,
        "report_to": [],   # 不上傳到 wandb
    }
}


# RUN

In [41]:
!pip install bitsandbytes==0.45.3

In [ ]:
!pip show bitsandbytes

Name: bitsandbytes
Version: 0.45.3
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/bitsandbytes-foundation/bitsandbytes
Author: 
Author-email: Tim Dettmers <dettmers@cs.washington.edu>
License: MIT License

Copyright (c) Facebook, Inc. and its affiliates.

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHA

In [56]:
train_sft(config)

Using output directory: /content/drive/My Drive/TAMU_LAB/experiences/models/SFT_run_colab_1
Torch CUDA: 12.6
Output dir: /content/drive/My Drive/TAMU_LAB/experiences/models/SFT_run_colab_1

Loading model: Qwen/Qwen3-0.6B
Loading datasets…
Applying LoRA…

Applying LoRA
trainable params: 2,293,760 || all params: 598,343,680 || trainable%: 0.3834
Commencing training…

Starting training...


Applying formatting function to train dataset:   0%|          | 0/1024 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/1024 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1024 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1024 [00:00<?, ? examples/s]

...

Truncating eval dataset:   0%|          | 0/256 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss,Validation Loss
100,2.392500,2.020748
200,1.876800,1.929409
300,2.009400,1.886618
400,1.764800,1.865339
500,1.849500,1.854947
600,1.934900,1.845719
700,1.715400,1.843758


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Training completed, model saved to: /content/drive/My Drive/TAMU_LAB/experiences/models/SFT_run_colab_1/final_model
